# Setup 

In [48]:
import os 
import pandas as pd 
import numpy as np 

import matplotlib.pyplot as plt 
from matplotlib.patches import Ellipse, Polygon, FancyArrowPatch  
from matplotlib.animation import FuncAnimation 
import plotly.graph_objects as go 


# get the folder path from an environment variable 
folder_path = os.environ.get("NFL_DATA_PATH") 

# Animation Calculations 

## process_keyframes 

In [2]:
# function to take a dictionary of keyframes and create a dataframe from it 
def process_keyframes(keyframes): 

    # initial dataframe 
    dfk = pd.DataFrame() 

    # loop through each frame and create a dataframe for it 
    for frame in keyframes:
        dfk = pd.concat([dfk, pd.DataFrame({
            "TIME": frame['time'], 
            "PARAM": frame['values'].keys(), 
            "VALUE": frame['values'].values() 
        })]) 
    
    # reset the index 
    dfk = dfk.reset_index(drop = True) 

    return dfk 

# some sample keyframes 
keyframes = [
    {
        "time": 0, 
        "values": {
            "xc": 30, 
            "yc": 90 
        }
    }, 
    {
        "time": 1, 
        "values": {
            "yc": 100 
        }
    }
] 

# showcase the function 
dfk = process_keyframes(keyframes) 
dfk 

,TIME,PARAM,VALUE
0,0,xc,30
1,0,yc,90
2,1,yc,100


## params_at_time 

In [3]:
# function to calculate parameter values at a given time 
def params_at_time(t, dfk): 

    # sort by time 
    dfk = dfk.sort_values(["TIME"]) 

    # get the parameter values before 
    df_before = (
        dfk[dfk["TIME"] <= t].groupby("PARAM").last() 
        .rename(columns = {"TIME": "TIME_BEFORE", "VALUE": "VALUE_BEFORE"}) 
    ) 

    # get the parameter values after
    df_after = (
        dfk[dfk["TIME"] > t].groupby("PARAM").first() 
        .rename(columns = {"TIME": "TIME_AFTER", "VALUE": "VALUE_AFTER"})
    ) 

    # merge the two and add the time 
    df_merged = df_before.merge(df_after, on = ["PARAM"], how = "outer").reset_index()  
    df_merged["TIME"] = t 

    # calculate the interpolated value
    def interp_value(row): 

        # if either is na, return the other 
        if pd.isna(row["VALUE_BEFORE"]): 
            return row["VALUE_AFTER"] 
        elif pd.isna(row["VALUE_AFTER"]): 
            return row["VALUE_BEFORE"] 

        # otherwise, do a linear interpolation 
        else: 
            t0 = row["TIME_BEFORE"] 
            t1 = row["TIME_AFTER"] 
            v0 = row["VALUE_BEFORE"] 
            v1 = row["VALUE_AFTER"] 
            return v0 + (v1 - v0) * ((t - t0) / (t1 - t0)) 
    
    # apply the interpolation function 
    df_merged["VALUE"] = df_merged.apply(interp_value, axis = 1) 

    # transform to a dictionary 
    pdict = {row["PARAM"]: row["VALUE"] for _, row in df_merged.iterrows()}

    return pdict 

# showcase the function 
pdict = params_at_time(3, dfk) 
pdict 

{'xc': 30, 'yc': 100}

## keyframes_to_params 

In [4]:
# function to transform keyframes into parameter values at a given time 
def keyframes_to_params(t, keyframes, defaults):

    # if no keyframes, just take defaults 
    if len(keyframes) == 0:
        pdict = defaults 

    # otherwise, process keyframes and fill in defaults 
    else:
        dfk = process_keyframes(keyframes) 
        pdict = params_at_time(t, dfk) 

        # fill in default parameters 
        for key, value in defaults.items(): 
            if key not in pdict: 
                pdict[key] = value 
    
    return pdict 

# showcase the function 
keyframes_to_params(
    t = 3, 
    keyframes = keyframes, 
    defaults = {
        "xc": 0, 
        "yc": 0, 
        "alpha": 1 
    }
) 

{'xc': 30, 'yc': 100, 'alpha': 1}

# Animate Play 

## PlayAnimation 

In [5]:
# class for each play animation 
class PlayAnimation:

    def __init__(self, ystart = 0, yend = 120):
        self.ystart = ystart 
        self.yend = yend 
        self.pixels_width = 1920 
        self.pixels_height = int(self.pixels_width * ((yend - ystart) / 53.3)) 
        self.dpi = 120 
        self.objects = {}  
    
    # create a figure object based on the sizing 
    def create_figure(self):
        return plt.Figure(
            figsize = (
                self.pixels_width / self.dpi, 
                self.pixels_height / self.dpi
            ), 
            dpi = self.dpi 
        ) 

    # create an axis object based on the sizing 
    def create_axis(self, fig, margin = 5):
        return fig.add_axes([
            margin / self.pixels_width, 
            margin / self.pixels_height, 
            1 - (2 * margin / self.pixels_width), 
            1 - (2 * margin / self.pixels_height)
        ]) 
    
    # add an object to the play 
    def add_object(self, obj):
        self.objects[obj.id] = obj 
    
    # add keyframes for the players 
    def add_keyframes(self, keyframes):
        for k, v in keyframes.items():
            if k in self.objects:
                self.objects[k].keyframes += v 

    # draw the play at the current time 
    def draw_play(self, fig, t = 0): 

        # create a new axis 
        ax = self.create_axis(fig) 

        # draw out the objects 
        for obj in self.objects.values():
            obj.draw(ax, t = t) 

        # Set limits and hide axes 
        ax.set_xlim(0, 53.3)
        ax.set_ylim(self.ystart, self.yend) 
        ax.axis('off') 
    
    # save an image of the play at time t 
    def save_image(self, t = 0, save_file = "output_test.png"):

        # draw the play 
        fig = self.create_figure() 
        self.draw_play(fig, t = t) 
        
        # save the figure output for testing 
        fig.savefig(save_file, dpi = self.dpi, pad_inches = 0)

    # create an animation of the play over time 
    def create_animation(self, end_time, fps = 12, save_file = "output_animation.mp4"):
        
        # calculate the times at each animation frame 
        tlist = np.linspace(0, end_time, int(np.max([np.round(end_time * fps), 1]))) 

        # create the figure and axis 
        fig = self.create_figure() 
        ax = self.create_axis(fig) 

        # create the animation function 
        def animate(i):
            if i % 20 == 1:
                print(f"  Creating frame {i} / {len(tlist)}...") 

            # clear out the previous frame 
            fig.clf() 
                
            # draw the play at time tlist[i] 
            self.draw_play(fig, t = tlist[i]) 
        
        # compile and save the animation 
        ani = FuncAnimation(fig, animate, frames = len(tlist) )
        ani.save(save_file, writer = 'ffmpeg', fps = fps) 

## Field 

### draw_field 

In [6]:
# function to draw the field background 
def draw_field(ax, alpha = 1): 

    # Set the background color to green
    ax.add_patch(plt.Rectangle(
        xy = (0, 0), 
        height = 120, 
        width = 53.3, 
        color = [0.32, 0.56, 0.23, alpha], 
        zorder = 0 
    )) 

    # add rectangles for the endzones 
    for i in [0, 110]:
        ax.add_patch(plt.Rectangle(
            xy = (0, i), 
            height = 10, 
            width = 53.3, 
            color = [0.22, 0.38, 0.16, alpha], 
            zorder = 1  
        )) 

    # draw a line for every 5 yards 
    for i in range(0, 21):
        ax.plot(
            [0, 53.3], 
            [10 + (i * 5)] * 2, 
            color = 'white', 
            linewidth = [4, 2][i % 2], 
            alpha = alpha, 
            zorder = 2 
        ) 

    # add the hash marks 
    for x in [20.65, 32.65]: # left and right hash marks 
        for y in range(0, 20):
            yc = (y * 5) + 10 
            ax.plot(
                [x] * 2, 
                [yc - 0.5, yc + 0.5], 
                color = 'white', 
                linewidth = 2, 
                alpha = alpha, 
                zorder = 2 
            ) 

    # add markers for each yard line 
    for x in [0.5, 20.65, 32.65, 52.8]:
        for yl in range(0, 100):
            ax.plot(
                [x - 0.5, x + 0.5], 
                [yl + 10] * 2, 
                color = 'white', 
                linewidth = 2, 
                alpha = alpha, 
                zorder = 2 
            ) 

    # add the numbers 
    for i, txt in enumerate(["1 0", "2 0", "3 0", "4 0", "5 0", "4 0", "3 0", "2 0", "1 0"]):
        for x in [8, 53.3 - 8]:
            ax.text(
                x = x, 
                y = (i * 10) + 20, 
                s = txt, 
                color = 'white', 
                fontsize = 44, 
                fontweight = 'bold', 
                horizontalalignment = 'center', 
                verticalalignment = 'center', 
                rotation = -90 if x < 26.65 else 90, 
                alpha = 0.65 * alpha, 
                zorder = 2 
            ) 

### FieldObject 

In [7]:
# class to represent the field background 
class FieldObject:

    def __init__(self, id = "field"):
        self.id = id 
        self.keyframes = [] 

        # default parameters 
        self.defaults = {
            "alpha": 1 
        }

    # draw the field background
    def draw(self, ax, t = 0):

        # calculate parameters at time t 
        pdict = keyframes_to_params(
            t = t, 
            keyframes = self.keyframes, 
            defaults = self.defaults 
        ) 

        # draw the field 
        draw_field(ax, alpha = pdict["alpha"]) 

# # create the play animation object 
# pa = PlayAnimation(ystart = 85, yend = 120) 

# # add the field object 
# pa.add_object(FieldObject()) 

# # test it out 
# pa.save_image() 
# plt.close() 

## Players 

### rotate_coords 

In [8]:
# function to rotate coordinates based on a given angle 
def rotate_coords(center, shift, angle_deg):
    angle_rad = np.radians(-angle_deg)
    return [
        (shift[0] * np.cos(angle_rad)) - (shift[1] * np.sin(angle_rad)) + center[0], 
        (shift[0] * np.sin(angle_rad)) + (shift[1] * np.cos(angle_rad)) + center[1] 
    ] 

# showcase the function 
rotate_coords(center = [10, 10], shift = [1, 0], angle_deg = 80) 

[np.float64(10.17364817766693), np.float64(9.015192246987793)]

### draw_player 

In [9]:
# function to draw out a player 
def draw_player(ax, x, y, angle = 0, pcolor = "red", alpha = 1):

    # draw the body ellipse 
    ax.add_patch(Ellipse(
        xy = (x, y), 
        width = 1.5, 
        height = 0.75, 
        angle = -angle, 
        facecolor = pcolor, 
        edgecolor = "black", 
        linewidth = 2,
        zorder = 25, 
        alpha = alpha 
    )) 

    # draw the head ellipse 
    ax.add_patch(Ellipse(
        xy = (x, y), 
        width = 0.5, 
        height = 0.5, 
        angle = -angle, 
        facecolor = pcolor, 
        edgecolor = "black", 
        linewidth = 2,
        zorder = 26, 
        alpha = alpha 
    )) 

    # trapezoid dimensions  
    dw1 = 0.25 
    dw2 = 1   
    dh = 0.75  

    # calculate rotated corners 
    corners = [[-dw1, 0], [-dw2, dh], [dw2, dh], [dw1, 0]] 
    rotated_corners = [rotate_coords(center = [x, y], shift = corner, angle_deg = angle) for corner in corners] 

    # draw the polygon 
    ax.add_patch(Polygon(
        rotated_corners, 
        facecolor = "yellow", 
        edgecolor = None, 
        alpha = 0.65 * alpha, 
        zorder = 24 
    )) 

### PlayerObject 

In [10]:
# class object to define each player 
class PlayerObject:

    def __init__(self, id, pcolor = "red"): 

        # initial parameters 
        self.id = id 
        self.pcolor = pcolor 
        self.keyframes = [] 

        # default parameters 
        self.defaults = {
            "xc": 0, 
            "yc": 0, 
            "angle": 0, 
            "alpha": 1 
        } 

    # draw the player at time t 
    def draw(self, ax, t = 0): 

        # calculate parameters at time t 
        pdict = keyframes_to_params(
            t = t, 
            keyframes = self.keyframes, 
            defaults = self.defaults 
        ) 

        # draw the player 
        draw_player(
            ax = ax, 
            x = pdict["xc"], 
            y = pdict["yc"], 
            angle = pdict["angle"], 
            pcolor = self.pcolor, 
            alpha = pdict["alpha"] 
        )

# create the play animation object with the field 
pa = PlayAnimation(ystart = 85, yend = 120) 
pa.add_object(FieldObject()) 

# add some players 
pa.add_object(PlayerObject(id = "1", pcolor = "blue")) 
pa.add_object(PlayerObject(id = "2", pcolor = "red")) 

# add keyframes for the players 
pa.add_keyframes({
    "1": [
        {
            "time": 0, 
            "values": {
                "xc": 30, 
                "yc": 90, 
                "angle": 0 
            }
        } 
    ],
    "2": [
        {
            "time": 0, 
            "values": {
                "xc": 35, 
                "yc": 94, 
                "angle": 240 
            }
        }
    ]
}) 

# draw and save the play 
pa.save_image() 
plt.close() 

## Routes 

### create_routes_df 

In [11]:
# function to create a dataframe from the given routes 
def create_routes_df(routes): 

    # initial dataframe 
    dfr = pd.DataFrame() 

    # loop through each route 
    for i, route in enumerate(routes):

        # create a dataframe for the route 
        dfr = pd.concat([dfr, pd.DataFrame({
            "TIME": [((frame - 1) / route["fps"]) + route["start_time"] for frame in route["path"]["frame_id"]],
            "XC": route["path"]["xc"], 
            "YC": route["path"]["yc"] 
        })], ignore_index=True) 
    
    # reset the index 
    dfr = dfr.sort_values("TIME").reset_index(drop = True) 

    # get the previous values 
    dfr["PREV_XC"] = dfr["XC"].shift(1)
    dfr["PREV_YC"] = dfr["YC"].shift(1)
    dfr["PREV_TIME"] = dfr["TIME"].shift(1) 

    return dfr 

# sample routes data 
routes = [
    {
        "start_time": 0, 
        "fps": 10, 
        "path": {
            "frame_id": [1, 2, 3, 4], 
            "xc": [1, 2, 3, 4], 
            "yc": [2, 4, 6, 8] 
        }
    }
] 

# test the function 
dfr = create_routes_df(routes) 
dfr 

,TIME,XC,YC,PREV_XC,PREV_YC,PREV_TIME
0,0.0,1,2,NaN,NaN,NaN
1,0.1,2,4,1.0,2.0,0.0
2,0.2,3,6,2.0,4.0,0.1
3,0.3,4,8,3.0,6.0,0.2


### calc_route_line 

In [12]:
# function to calculate a line at the current time 
def calc_route_line(t, dfr): 

    # loop through each row and add to the arrays 
    xlist = [] 
    ylist = [] 
    for i, row in dfr.iterrows():

        # if time is before or at the current time, add the point 
        if row["TIME"] <= t:
            xlist.append(row["XC"]) 
            ylist.append(row["YC"]) 

        # if time is after the current time, interpolate and break 
        elif row["PREV_TIME"] < t and pd.notna(row["PREV_TIME"]):
            t0 = row["PREV_TIME"] 
            t1 = row["TIME"] 
            x0 = row["PREV_XC"] 
            x1 = row["XC"] 
            y0 = row["PREV_YC"] 
            y1 = row["YC"] 
            x_interp = x0 + (x1 - x0) * ((t - t0) / (t1 - t0)) 
            y_interp = y0 + (y1 - y0) * ((t - t0) / (t1 - t0)) 
            xlist.append(x_interp) 
            ylist.append(y_interp) 
            break 
    
    return xlist, ylist

# test the function 
calc_route_line(0.38, dfr)

([np.float64(1.0), np.float64(2.0), np.float64(3.0), np.float64(4.0)],
 [np.float64(2.0), np.float64(4.0), np.float64(6.0), np.float64(8.0)])

### RouteObject 

In [13]:
# class for player routes/paths 
class RouteObject:

    def __init__(self, id, routes = [], rcolor = "blue"): 

        # initial parameters 
        self.id = id 
        self.rcolor = rcolor 
        self.keyframes = [] 
        self.routes = routes

        # default parameters 
        self.defaults = { 
            "alpha": 1 
        } 
    
    # draw the route at time t 
    def draw(self, ax, t = 0): 

        # calulate parameters at time t 
        pdict = keyframes_to_params(
            t = t, 
            keyframes = self.keyframes, 
            defaults = self.defaults 
        ) 

        # calculate route path at time t 
        dfr = create_routes_df(self.routes) 
        xlist, ylist = calc_route_line(t, dfr) 

        # draw the route
        ax.plot(
            xlist, 
            ylist, 
            linewidth = 4, 
            color = self.rcolor, 
            alpha = self.defaults["alpha"], 
            zorder = 15 
        ) 

# # create the play animation with the field 
# pa = PlayAnimation(ystart = 85, yend = 120) 
# pa.add_object(FieldObject()) 

# # add a route object 
# pa.add_object(RouteObject(
#     id = "route_1", 
#     routes = [
#         {
#             "start_time": 0, 
#             "fps": 10, 
#             "path": {
#                 "frame_id": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 
#                 "xc": [20, 21, 22, 23, 24, 24, 24, 23, 22, 21], 
#                 "yc": [90, 91, 92, 93, 94, 95, 96, 97, 98, 99] 
#             }
#         }
#     ], 
#     rcolor = "blue"
# )) 

# # draw and save the play 
# pa.save_image(t = 0.85) 
# plt.close() 

## Metrics 

### MetricObject 

In [14]:
# class object to display a metric 
class MetricObject:

    def __init__(self, id, suffix = "", tcolor = "black"):

        # initial parameters 
        self.id = id 
        self.suffix = suffix 
        self.tcolor = tcolor
        self.keyframes = [] 

        # default parameters 
        self.defaults = {
            "xc": 0, 
            "yc": 0, 
            "value": 0, 
            "alpha": 1 
        } 

    # draw the metric at time t 
    def draw(self, ax, t = 0): 

        # calculate parameters at time t 
        pdict = keyframes_to_params(
            t = t, 
            keyframes = self.keyframes, 
            defaults = self.defaults 
        ) 

        # draw the metric text 
        ax.text(
            x = pdict["xc"], 
            y = pdict["yc"], 
            s = f"{pdict['value']:.1f}{self.suffix}", 
            color = self.tcolor, 
            fontsize = 28, 
            fontweight = 'bold', 
            horizontalalignment = 'center', 
            verticalalignment = 'center', 
            alpha = pdict["alpha"], 
            zorder = 30 
        ) 

## Football 

### draw_football

In [15]:
# function to draw out the football 
def draw_football(ax, x, y, angle = 0, alpha = 1):

    # draw the ellipse 
    ax.add_patch(Ellipse(
        xy = (x, y), 
        width = 0.5, 
        height = 1.0, 
        angle = -angle, 
        facecolor = "brown", 
        edgecolor = "black", 
        linewidth = 2,
        zorder = 32, 
        alpha = alpha 
    )) 

### FootballObject 

In [16]:
# class object to define the football 
class FootballObject:

    def __init__(self, id): 

        # initial parameters 
        self.id = id 
        self.keyframes = [] 

        # default parameters 
        self.defaults = {
            "xc": 0, 
            "yc": 0, 
            "angle": 0, 
            "alpha": 1 
        } 

    # draw the football at time t 
    def draw(self, ax, t = 0): 

        # calculate parameters at time t 
        pdict = keyframes_to_params(
            t = t, 
            keyframes = self.keyframes, 
            defaults = self.defaults 
        ) 

        # draw the football 
        draw_football(
            ax = ax, 
            x = pdict["xc"], 
            y = pdict["yc"], 
            angle = pdict["angle"], 
            alpha = pdict["alpha"] 
        )

# # create the play animation object with the field 
# pa = PlayAnimation(ystart = 85, yend = 120) 
# pa.add_object(FieldObject()) 

# # add some players 
# pa.add_object(FootballObject(id = "ball")) 

# # add keyframes for the players 
# pa.add_keyframes({
#     "ball": [
#         {
#             "time": 0, 
#             "values": {
#                 "xc": 30, 
#                 "yc": 100, 
#                 "angle": 0 
#             }
#         } 
#     ] 
# }) 

# # draw and save the play 
# pa.save_image() 
# plt.close() 

## Line 

### LineSegmentObject 

In [54]:
# class for a simple line object 
class LineSegmentObject:

    def __init__(self, id, lcolor = "black", linewidth = 2, linestyle = "-", add_arrow = False):

        # initial parameters 
        self.id = id 
        self.lcolor = lcolor 
        self.linewidth = linewidth 
        self.linestyle = linestyle 
        self.add_arrow = add_arrow 
        self.keyframes = [] 

        # default parameters 
        self.defaults = {
            "x0": 0, 
            "y0": 0, 
            "x1": 1, 
            "y1": 1, 
            "alpha": 1 
        } 

    # draw the line at time t 
    def draw(self, ax, t = 0): 

        # calculate parameters at time t 
        pdict = keyframes_to_params(
            t = t, 
            keyframes = self.keyframes, 
            defaults = self.defaults 
        ) 

        # draw the line with or without arrow
        if self.add_arrow:
            arrow = FancyArrowPatch(
                (pdict["x0"], pdict["y0"]),
                (pdict["x1"], pdict["y1"]),
                color = self.lcolor,
                linewidth = self.linewidth,
                linestyle = self.linestyle,
                alpha = pdict["alpha"],
                arrowstyle = '->', # or use '-|>' for a filled arrow
                mutation_scale = 10 * self.linewidth, # controls arrow head size
                zorder = 10
            )
            ax.add_patch(arrow)
        else:
            ax.plot(
                [pdict["x0"], pdict["x1"]],
                [pdict["y0"], pdict["y1"]], 
                color = self.lcolor, 
                linewidth = self.linewidth, 
                linestyle = self.linestyle, 
                alpha = pdict["alpha"], 
                zorder = 10 
            )

## Quick Demo

In [18]:
# # create the play animation with the field 
# pa = PlayAnimation(ystart = 85, yend = 120) 
# pa.add_object(FieldObject()) 

# # add some players 
# pa.add_object(PlayerObject(id = "1", pcolor = "blue")) 
# pa.add_object(PlayerObject(id = "2", pcolor = "red")) 

# # add keyframes for the players 
# pa.add_keyframes({
#     "1": [
#         {
#             "time": 0, 
#             "values": {
#                 "xc": 22, 
#                 "yc": 90, 
#                 "angle": -80 
#             }
#         }, 
#         {
#             "time": 2, 
#             "values": {
#                 "xc": 17, 
#                 "yc": 95, 
#                 "angle": -20 
#             }
#         } 
#     ], 
#     "2": [
#         {
#             "time": 0, 
#             "values": {
#                 "xc": 25, 
#                 "yc": 100, 
#                 "angle": -170 
#             }
#         }, 
#         {
#             "time": 2, 
#             "values": {
#                 "xc": 18, 
#                 "yc": 96, 
#                 "angle": -100 
#             }
#         } 
#     ]
# }) 

# # add a route for the blue player 
# pa.add_object(RouteObject(
#     id = "route_1", 
#     routes = [
#         {
#             "start_time": 0, 
#             "fps": 0.5, 
#             "path": {
#                 "frame_id": [1, 2], 
#                 "xc": [22, 17], 
#                 "yc": [90, 95] 
#             }
#         }
#     ], 
#     rcolor = "blue"
# )) 

# # create and save the animation 
# pa.create_animation(
#     end_time = 2.5, 
#     fps = 12, 
#     save_file = "test_animation.mp4"
# ) 
# plt.close() 

# Add Play Data 

In [19]:
# read in the play metadata
df_meta = pd.read_csv(f"{folder_path}/supplementary_data.csv") 

# showcase the data 
df_meta.head() 

C:\Users\rjfis\AppData\Local\Temp\ipykernel_22536\1083874482.py:2: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  df_meta = pd.read_csv(f"{folder_path}/supplementary_data.csv")


,game_id,season,week,game_date,game_time_eastern,home_team_abbr,visitor_team_abbr,play_id,play_description,quarter,...,team_coverage_type,penalty_yards,pre_penalty_yards_gained,yards_gained,expected_points,expected_points_added,pre_snap_home_team_win_probability,pre_snap_visitor_team_win_probability,home_team_win_probability_added,visitor_team_win_probility_added
0,2023090700,2023,1,09/07/2023,20:20:00,KC,DET,3461,(10:46) (Shotgun) J.Goff pass deep left to J.R...,4,...,COVER_2_ZONE,NaN,18,18,-0.664416,2.945847,0.834296,0.165704,-0.081149,0.081149
1,2023090700,2023,1,09/07/2023,20:20:00,KC,DET,461,(7:30) J.Goff pass short right to J.Reynolds t...,1,...,COVER_6_ZONE,NaN,21,21,1.926131,1.345633,0.544618,0.455382,-0.029415,0.029415
2,2023090700,2023,1,09/07/2023,20:20:00,KC,DET,1940,(:09) (Shotgun) J.Goff pass incomplete deep ri...,2,...,COVER_2_ZONE,NaN,0,0,0.281891,-0.081964,0.771994,0.228006,0.000791,-0.000791
3,2023090700,2023,1,09/07/2023,20:20:00,KC,DET,1711,"(:45) (No Huddle, Shotgun) P.Mahomes pass deep...",2,...,COVER_2_ZONE,NaN,26,26,3.452352,2.342947,0.663187,0.336813,0.041843,-0.041843
4,2023090700,2023,1,09/07/2023,20:20:00,KC,DET,1588,(1:54) (Shotgun) P.Mahomes pass incomplete dee...,2,...,COVER_4_ZONE,NaN,0,0,1.921525,-0.324035,0.615035,0.384965,0.000061,-0.000061


## Transform Parameters 

In [20]:
# function to transform the x coordinate 
def calc_new_x(y, flip_y = False):
    xnew = 53.3 - y 

    # flip if needed 
    if flip_y:
        xnew = 53.3 - xnew 

    return xnew 

# function to transform the y coordinate 
def calc_new_y(x, flip_y = False):
    ynew = x 

    # flip if needed 
    if flip_y:
        ynew = 120 - ynew 

    return ynew 

# function to transform the angle 
def calc_new_angle(angle, flip_y = False):
    anglenew = angle - 90

    # flip if needed 
    if flip_y:
        anglenew = (anglenew + 180) % 360 

    return anglenew  

## PlayDataLoader 

In [21]:
# class to load in and transform the data for a given play 
class PlayDataLoader: 

    def __init__(self, game_id, play_id, df_meta): 
        self.game_id = game_id 
        self.play_id = play_id 

        # filter the metadata for the given play 
        self.play_meta = df_meta.loc[
            (df_meta["game_id"] == game_id) & 
            (df_meta["play_id"] == play_id)
        ].reset_index(drop = True).to_dict(orient = "records")[0] 

        # load in the input and output data 
        df_input = pd.read_csv(f"{folder_path}//train//input_2023_w{self.play_meta['week']:02}.csv") 
        df_output = pd.read_csv(f"{folder_path}//train//output_2023_w{self.play_meta['week']:02}.csv") 

        # filter for the given play 
        df_input = df_input.loc[
            (df_input["game_id"] == game_id) & 
            (df_input["play_id"] == play_id)
        ].reset_index(drop = True) 
        df_output = df_output.loc[
            (df_output["game_id"] == game_id) & 
            (df_output["play_id"] == play_id) 
        ].reset_index(drop = True) 

        # add some of the input attributes for the play 
        add_meta = df_input[["play_direction", "ball_land_x", "ball_land_y"]].to_dict(orient = "records")[0] 
        self.play_meta.update({
            "play_direction": add_meta["play_direction"], 
            "ball_land_x": calc_new_x(add_meta["ball_land_y"], flip_y = add_meta["play_direction"] == "left"), 
            "ball_land_y": calc_new_y(add_meta["ball_land_x"], flip_y = add_meta["play_direction"] == "left") 
        }) 

        # get a unique dataframe of players in the play 
        self.df_players = (
            df_input[["nfl_id", "player_to_predict", "player_side", "player_position", "player_name"]]
            .drop_duplicates().reset_index(drop = True) 
        ) 

        # get the number of input and output frames 
        self.input_frames = df_input["frame_id"].max() 
        self.output_frames = df_output["frame_id"].max() 
        self.total_frames = self.input_frames + self.output_frames 

        # add the columns to union the two dataframes 
        df_input["source"] = "input" 
        df_output["source"] = "output" 
        df_input["orig_frame_id"] = df_input["frame_id"] 
        df_output["orig_frame_id"] = df_output["frame_id"] 
        df_output["frame_id"] = df_output["frame_id"] + self.input_frames 
        # df_output["o"] = 9999  # placeholder for orientation in output data 

        # put the position metrics together 
        df_pos = pd.concat([
            df_input[["nfl_id", "frame_id", "x", "y", "o", "orig_frame_id", "source"]], 
            df_output[["nfl_id", "frame_id", "x", "y", "orig_frame_id", "source"]] 
        ]).sort_values(["nfl_id", "frame_id"]).reset_index(drop = True) 

        # calculate the transformed coordinates and angle 
        flip_y = self.play_meta["play_direction"] == "left" 
        df_pos["x_new"] = calc_new_x(df_pos["y"], flip_y = flip_y) 
        df_pos["y_new"] = calc_new_y(df_pos["x"], flip_y = flip_y) 
        df_pos["o_new"] = calc_new_angle(df_pos["o"], flip_y = flip_y) 
        df_pos = df_pos.drop(columns = ["x", "y", "o"]).rename(columns = {
            "x_new": "x", 
            "y_new": "y",
            "o_new": "o" 
        })

        # calculate speed and acceleration 
        df_pos["x_prev"] = df_pos.groupby("nfl_id")["x"].shift(1) 
        df_pos["y_prev"] = df_pos.groupby("nfl_id")["y"].shift(1) 
        df_pos["distance"] = np.sqrt((df_pos["x"] - df_pos["x_prev"])**2 + (df_pos["y"] - df_pos["y_prev"])**2) * 10 
        df_pos["speed"] = df_pos["distance"]  # speed in yards per second (assuming 10 fps)
        df_pos["speed_prev"] = df_pos.groupby("nfl_id")["speed"].shift(1) 
        df_pos["acceleration"] = (df_pos["speed"] - df_pos["speed_prev"]) * 10  # acceleration in yards per second squared 
        df_pos["speed_mph"] = df_pos["speed"] * (3600 / 1760)  # convert speed to miles per hour 

        # calculate orientation for output frames 
        df_pos["o_calc"] = np.degrees(np.arctan2(
            df_pos["x"] - df_pos["x_prev"], 
            df_pos["y"] - df_pos["y_prev"] 
        )) % 360 
        df_pos["o"] = df_pos["o"].fillna(df_pos["o_calc"]) 

        # calculate distance to the ball landing position 
        df_pos["dist_to_ball"] = np.sqrt(
            (df_pos["x"] - self.play_meta["ball_land_x"])**2 + 
            (df_pos["y"] - self.play_meta["ball_land_y"])**2
        ) 

        # calculate acceleration to the ball landing position 
        df_pos["dist_to_ball_prev"] = df_pos.groupby("nfl_id")["dist_to_ball"].shift(1) 
        df_pos["speed_to_ball"] = (df_pos["dist_to_ball_prev"] - df_pos["dist_to_ball"]) * 10  # speed in yards per second 
        df_pos["speed_to_ball_prev"] = df_pos.groupby("nfl_id")["speed_to_ball"].shift(1) 
        df_pos["accel_to_ball"] = (df_pos["speed_to_ball"] - df_pos["speed_to_ball_prev"]) * 10  # acceleration in yards per second squared 

        # add as a dataframe attribute 
        self.df_pos = df_pos 
    
    # method to get the data for a specific player 
    def get_player_data(self, nfl_id, from_frame = None, to_frame = None):

        # filter for the given player 
        df_player = self.df_pos.loc[self.df_pos["nfl_id"] == nfl_id].reset_index(drop = True) 

        # filter for the given frame range 
        if from_frame is not None:
            df_player = df_player.loc[df_player["frame_id"] >= from_frame].reset_index(drop = True)
        if to_frame is not None:
            df_player = df_player.loc[df_player["frame_id"] <= to_frame].reset_index(drop = True) 
        
        return df_player 


# # test out the class 
# pdl = PlayDataLoader(
#     game_id = 2023120400, 
#     play_id = 2958, 
#     df_meta = df_meta 
# ) 
# pdl.df_pos.head() 

## create_player_keyframes 

In [22]:
# function to create keyframes for coordinates and orientation of a given player 
def create_player_keyframes(df_pos, fps = 10, start_time = 0): 

    # initial keyframes list 
    keyframes = [] 

    # loop through each row and create a keyframe 
    for i, row in df_pos.iterrows():
        keyframe = {
            "time": ((row["frame_id"] - 1) / fps) + start_time, 
            "values": {
                "xc": row["x"], 
                "yc": row["y"], 
                "angle": row["o"] 
            }
        } 
        keyframes.append(keyframe) 
    
    return keyframes 

## create_metric_keyframes 

In [23]:
# function to create keyframes for a given metric 
def create_metric_keyframes(df_pos, metric_col, xoff = 0, yoff = 0, fps = 10, start_time = 0): 

    # initial keyframes list 
    keyframes = [] 

    # loop through each row and create a keyframe 
    for i, row in df_pos.iterrows():
        keyframe = {
            "time": ((row["frame_id"] - 1) / fps) + start_time, 
            "values": {
                "xc": row["x"] + xoff, 
                "yc": row["y"] + yoff, 
                "value": row[metric_col] 
            }
        } 
        keyframes.append(keyframe) 
    
    return keyframes 

## create_route_definition 

In [24]:
def create_route_definition(df_pos, fps = 10, start_time = 0): 

    # get the min frame id 
    first_frame = df_pos["frame_id"].min() 

    # create the route definition 
    route_def = {
        "start_time": start_time, 
        "fps": fps, 
        "path": {
            "frame_id": (df_pos["frame_id"] - first_frame + 1).tolist(), 
            "xc": df_pos["x"].tolist(), 
            "yc": df_pos["y"].tolist() 
        }
    } 

    return route_def 

## create_ball_keyframes 

In [25]:
def create_ball_keyframes(pdl):

    # fade in the ball at the start of the play 
    keyframes = [
        {
            "time": 0, 
            "values": {
                "alpha": 0 
            }
        }, 
        {
            "time": 0.2, 
            "values": {
                "alpha": 1 
            }
        } 
    ]

    # get the player data for the QB 
    qb_id = pdl.df_players.loc[pdl.df_players["player_position"] == "QB"]["nfl_id"].tolist()[0] 
    df_qb = pdl.get_player_data(nfl_id = qb_id, to_frame = pdl.input_frames) 

    # create keyframes for when the ball is with the QB 
    keyframes += create_player_keyframes(
        df_pos = df_qb,
        fps = 10,
        start_time = 0
    ) 

    # get the end position of the QB when the ball is thrown 
    qb_end = df_qb.loc[df_qb["frame_id"] == pdl.input_frames].reset_index(drop = True).iloc[0] 

    # get the angle that the ball is traveling while in the air 
    ball_angle = np.degrees(np.arctan2(
        pdl.play_meta["ball_land_x"] - qb_end["x"], 
        pdl.play_meta["ball_land_y"] - qb_end["y"] 
    )) % 360 

    # add a keyframe for the ball angle while in the air 
    keyframes += [{
        "time": (pdl.input_frames / 10) + 0.1, 
        "values": {
            "angle": ball_angle
        } 
    }] 

    # add a keyframe for the ball landing position 
    keyframes += [{
        "time": (pdl.total_frames / 10), 
        "values": {
            "xc": pdl.play_meta["ball_land_x"], 
            "yc": pdl.play_meta["ball_land_y"]
        } 
    }] 

    return keyframes 

# Play Examples 

## setup_play_example 

In [26]:
# function to showcase a given play with routes 
def setup_play_example(game_id, play_id, add_players = True, player_routes = []): 

    # load the play data 
    pdl = PlayDataLoader(
        game_id = game_id, 
        play_id = play_id, 
        df_meta = df_meta 
    ) 

    # create the play animation with the field 
    pa = PlayAnimation(
        ystart = pdl.df_pos["y"].min() - 5,
        yend = pdl.df_pos["y"].max() + 5 
    ) 
    pa.add_object(FieldObject()) 
        
    # add the ball 
    pa.add_object(FootballObject(id = "Football")) 
    ball_keyframes = create_ball_keyframes(pdl) 
    pa.add_keyframes({ "Football": ball_keyframes }) 
        
    # add each of the players 
    if add_players:
        for i, row in pdl.df_players.iterrows():
            nfl_id = row["nfl_id"] 

            # get the player data 
            df_player = pdl.get_player_data(nfl_id = nfl_id) 

            # create the player object 
            pcolors = {"Offense": "blue", "Defense": "red"}
            player_obj = PlayerObject(id = f"Player {nfl_id}", pcolor = pcolors[row["player_side"]]) 

            # add the player to the play animation 
            pa.add_object(player_obj) 

            # create and add the keyframes 
            keyframes = create_player_keyframes(df_player, fps = 10, start_time = 0) 
            pa.add_keyframes({ f"Player {nfl_id}": keyframes }) 

            # add routes for the two players that we want to focus on 
            if nfl_id in player_routes:
                pa.add_object(RouteObject(
                    id = f"Route {nfl_id}", 
                    routes = [create_route_definition(df_player, fps = 10, start_time = 0)], 
                    rcolor = "#595959"
                )) 

    return pdl, pa 

## Example 1 

### Field Only 

In [27]:
# setup the play example 
pdl, pa = setup_play_example(
    game_id = 2023120400, 
    play_id = 2958, 
    add_players = True
) 

# save an image of just the field  
pa.save_image(save_file = f"{folder_path}//example_1_field.png") 

### Start and End Screenshots 

In [28]:
# setup the play example 
pdl, pa = setup_play_example(
    game_id = 2023120400, 
    play_id = 2958, 
    player_routes = [55925, 44849] 
) 

# save a screenshot of the start of the play 
pa.save_image(t = 0, save_file = f"{folder_path}//example_1_start.png") 

# save a screenshot of the end of the play 
end_time = pdl.total_frames / 10 
pa.save_image(t = end_time, save_file = f"{folder_path}//example_1_end.png") 

### Full Animation 

In [29]:
# setup the play example 
pdl, pa = setup_play_example(
    game_id = 2023120400, 
    play_id = 2958, 
    player_routes = [55925, 44849] 
) 

# players to add metrics for 
pvals = {
    55925: {
        "color": "red", 
        "xoff": 4, 
        "yoff": 2 
    }, 
    44849: {
        "color": "blue",
        "xoff": 2,
        "yoff": -3
    } 
} 

# add speed metrics for the two players 
for nfl_id, vals in pvals.items():

    # get the player data 
    df_player = pdl.get_player_data(nfl_id = nfl_id) 

    # add the speed metrics 
    pa.add_object(MetricObject(
        id = f"Speed {nfl_id}", 
        suffix = " mph", 
        tcolor = pvals[nfl_id]["color"] 
    )) 
    speed_keyframes = create_metric_keyframes(
        df_pos = df_player, 
        metric_col = "speed_mph", 
        xoff = pvals[nfl_id]["xoff"], 
        yoff = pvals[nfl_id]["yoff"]  
    ) 
    pa.add_keyframes({ f"Speed {nfl_id}": speed_keyframes }) 

# # draw the final animation 
# end_time = (pdl.total_frames / 10) + 2 
# pa.create_animation(
#     end_time = end_time, 
#     fps = 12, 
#     save_file = f"{folder_path}//example_1_animation.mp4"
# ) 

## Example 2 

### Field Only 

In [30]:
# setup the play example 
pdl, pa = setup_play_example(
    game_id = 2023092100, 
    play_id = 353, 
    add_players = False 
) 

# save an image of just the field  
pa.save_image(save_file = f"{folder_path}//example_2_field.png") 

### Start and End Screenshots 

In [31]:
# setup the play example 
pdl, pa = setup_play_example(
    game_id = 2023092100, 
    play_id = 353, 
    player_routes = [55888, 47819] 
) 

# save a screenshot of the start of the play 
pa.save_image(t = 0, save_file = f"{folder_path}//example_2_start.png") 

# save a screenshot of the end of the play 
end_time = pdl.total_frames / 10 
pa.save_image(t = end_time, save_file = f"{folder_path}//example_2_end.png") 

### Full Animation 

In [32]:
# setup the play example 
pdl, pa = setup_play_example(
    game_id = 2023092100, 
    play_id = 353, 
    player_routes = [55888, 47819] 
) 

# players to add metrics for 
pvals = {
    55888: {
        "color": "red", 
        "xoff": 5, 
        "yoff": -1 
    }, 
    47819: {
        "color": "blue",
        "xoff": 1,
        "yoff": 2
    } 
} 

# add speed metrics for the two players 
for nfl_id, vals in pvals.items():

    # get the player data 
    df_player = pdl.get_player_data(nfl_id = nfl_id) 

    # add the speed metrics 
    pa.add_object(MetricObject(
        id = f"Accel {nfl_id}", 
        suffix = " yd/s²", 
        tcolor = pvals[nfl_id]["color"]
    )) 
    speed_keyframes = create_metric_keyframes(
        df_pos = df_player, 
        metric_col = "accel_to_ball", 
        xoff = pvals[nfl_id]["xoff"], 
        yoff = pvals[nfl_id]["yoff"]  
    ) 
    pa.add_keyframes({ f"Accel {nfl_id}": speed_keyframes }) 

    # add line to ball landing position 
    pa.add_object(LineSegmentObject(
        id = f"Line to Ball {nfl_id}",
        lcolor = "#0d0d0d", 
        linewidth = 1, 
        linestyle = "--" 
    )) 

    # create keyframes for the line 
    line_keyframes = [
        {
            "time": (pdl.input_frames / 10), 
            "values": {
                "alpha": 0 
            }
        }, 
        {
            "time": (pdl.input_frames / 10) + 0.1,
            "values": {
                "alpha": 1 
            } 
        }
    ] 
    for i, row in df_player.iterrows():
        line_keyframe = {
            "time": ((row["frame_id"] - 1) / 10), 
            "values": {
                "x0": row["x"], 
                "y0": row["y"], 
                "x1": pdl.play_meta["ball_land_x"], 
                "y1": pdl.play_meta["ball_land_y"] 
            }
        } 
        line_keyframes.append(line_keyframe) 
    pa.add_keyframes({ f"Line to Ball {nfl_id}": line_keyframes }) 

    # create keyframes to fade the metrics in 
    pa.add_keyframes({
        f"Accel {nfl_id}": [
            {
                "time": (pdl.input_frames / 10), 
                "values": {
                    "alpha": 0 
                }
            }, 
            {
                "time": (pdl.input_frames / 10) + 0.1, 
                "values": {
                    "alpha": 1 
                }
            }
        ]
    }) 

# # draw the final animation 
# end_time = (pdl.total_frames / 10) + 2 
# pa.create_animation(
#     end_time = end_time, 
#     fps = 12, 
#     save_file = f"{folder_path}//example_2_animation.mp4"
# ) 

## Example 3 

In [55]:
# setup the play example 
pdl, pa = setup_play_example(
    game_id = 2023121700, 
    play_id = 878, 
    player_routes = [53458, 41233] 
) 

# save a screenshot of the start of the play 
pa.save_image(t = 0, save_file = f"{folder_path}//example_3_start.png") 

# add a line for the receiver's go route 
dfp = pdl.get_player_data(nfl_id = 41233) 
pos = dfp.loc[dfp["frame_id"] == 1].to_dict(orient = "records")[0] 
pa.add_object(LineSegmentObject(
    id = "Go Route Line", 
    lcolor = "blue", 
    linewidth = 4, 
    linestyle = "-", 
    add_arrow = True 
))
pa.add_keyframes({
    "Go Route Line": [
        {
            "time": 0, 
            "values": {
                "x0": pos["x"], 
                "y0": pos["y"], 
                "x1": pos["x"], 
                "y1": pos["y"] + 15
            }
        }
    ]
}) 

# save a screenshot with the go route line 
pa.save_image(t = 0, save_file = f"{folder_path}//example_3_route.png") 

# Animate Charts 

## ChartAnimation 

In [36]:
# class for each chart animation 
class ChartAnimation:

    def __init__(self, ymin = 0, xmax = 1, ymax = 1, plot_title = "Chart Title", x_label = "X Axis", y_label = "Y Axis"):
        self.pixels_width = 960  
        self.pixels_height = 540 
        self.dpi = 120 
        self.objects = {} 
        
        # store other info 
        self.xmax = xmax 
        self.ymin = ymin  
        self.ymax = ymax 
        self.plot_title = plot_title
        self.x_label = x_label
        self.y_label = y_label
    
    # create a figure object based on the sizing 
    def create_figure(self):
        return plt.Figure(
            figsize = (
                self.pixels_width / self.dpi, 
                self.pixels_height / self.dpi
            ), 
            dpi = self.dpi 
        ) 

    # create an axis object based on the sizing 
    def create_axis(self, fig, margin = 80):
        # Increased margin to accommodate axis labels and title
        return fig.add_axes([
            (margin + 10) / self.pixels_width, 
            margin / self.pixels_height, 
            1 - (2 * margin / self.pixels_width), 
            1 - (1.7 * margin / self.pixels_height)  # Extra space at top for title
        ]) 
    
    # add an object to the chart 
    def add_object(self, obj):
        self.objects[obj.id] = obj 

    # draw the chart at the current time 
    def draw_chart(self, fig, t = 0): 

        # create a new axis 
        ax = self.create_axis(fig) 

        # draw out the objects 
        for obj in self.objects.values():
            obj.draw(ax, t = t) 

        # Set limits
        ax.set_xlim(0, self.xmax) 
        ax.set_ylim(self.ymin, self.ymax) 
        
        # Add titles and labels
        ax.set_title(self.plot_title, fontsize=14, fontweight='bold', pad = 12)
        ax.set_xlabel(self.x_label, fontsize=12, fontweight='bold')
        ax.set_ylabel(self.y_label, fontsize=12, fontweight='bold')
        
        # Style the axis
        ax.grid(True, alpha=0.3, linestyle='--', linewidth=0.5)
        ax.tick_params(axis='both', which='major', labelsize=10)
        
        # Show only left and bottom spines (x and y axis lines)
        ax.spines['left'].set_visible(True)
        ax.spines['left'].set_edgecolor('black')
        ax.spines['left'].set_linewidth(1.5)
        
        ax.spines['bottom'].set_visible(True)
        ax.spines['bottom'].set_edgecolor('black')
        ax.spines['bottom'].set_linewidth(1.5)
        
        # Hide top and right spines
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False) 
    
    # save an image of the play at time t 
    def save_image(self, t = 0, save_file = "output_test.png"):

        # draw the play 
        fig = self.create_figure() 
        self.draw_chart(fig, t = t) 
        
        # save the figure output for testing 
        fig.savefig(save_file, dpi = self.dpi, pad_inches = 2)

    # create an animation of the play over time 
    def create_animation(self, end_time, fps = 12, save_file = "output_animation.mp4"):
        
        # calculate the times at each animation frame 
        tlist = np.linspace(0, end_time, int(np.max([np.round(end_time * fps), 1]))) 

        # create the figure and axis 
        fig = self.create_figure() 
        ax = self.create_axis(fig) 

        # create the animation function 
        def animate(i):
            if i % 20 == 1:
                print(f"  Creating frame {i} / {len(tlist)}...") 

            # clear out the previous frame 
            fig.clf() 
                
            # draw the chart at time tlist[i] 
            self.draw_chart(fig, t = tlist[i]) 
        
        # compile and save the animation 
        ani = FuncAnimation(fig, animate, frames = len(tlist) )
        ani.save(save_file, writer = 'ffmpeg', fps = fps) 

# ca = ChartAnimation(
#     xmax = 10, 
#     ymax = 100, 
#     plot_title = "Sample Chart",
#     x_label = "Time (seconds)",
#     y_label = "Value"
# ) 
# ca.save_image(save_file = f"{folder_path}//chart_test_image.png") 

## create_lines_df 

In [37]:
# function to create a dataframe from the given lines 
def create_lines_df(lines): 

    # initial dataframe 
    dfr = pd.DataFrame() 

    # loop through each line 
    for i, line in enumerate(lines):

        # create a dataframe for the line 
        dfr = pd.concat([dfr, pd.DataFrame({
            "TIME": [((frame - 1) / line["fps"]) + line["start_time"] for frame in line["path"]["frame_id"]],
            "XC": line["path"]["xc"], 
            "YC": line["path"]["yc"] 
        })], ignore_index=True) 
    
    # reset the index 
    dfr = dfr.sort_values("TIME").reset_index(drop = True) 

    # get the previous values 
    dfr["PREV_XC"] = dfr["XC"].shift(1)
    dfr["PREV_YC"] = dfr["YC"].shift(1)
    dfr["PREV_TIME"] = dfr["TIME"].shift(1) 

    return dfr 

# sample lines data 
lines = [
    {
        "start_time": 0, 
        "fps": 10, 
        "path": {
            "frame_id": [1, 2, 3, 4], 
            "xc": [1, 2, 3, 4], 
            "yc": [10, 15, 25, 26] 
        }
    }
] 

# test the function 
dfr = create_lines_df(lines) 
dfr 

,TIME,XC,YC,PREV_XC,PREV_YC,PREV_TIME
0,0.0,1,10,NaN,NaN,NaN
1,0.1,2,15,1.0,10.0,0.0
2,0.2,3,25,2.0,15.0,0.1
3,0.3,4,26,3.0,25.0,0.2


## calc_line_path 

In [38]:
# function to calculate a line at the current time 
def calc_line_path(t, dfr): 

    # loop through each row and add to the arrays 
    xlist = [] 
    ylist = [] 
    for i, row in dfr.iterrows():

        # if time is before or at the current time, add the point 
        if row["TIME"] <= t:
            xlist.append(row["XC"]) 
            ylist.append(row["YC"]) 

        # if time is after the current time, interpolate and break 
        elif row["PREV_TIME"] < t and pd.notna(row["PREV_TIME"]):
            t0 = row["PREV_TIME"] 
            t1 = row["TIME"] 
            x0 = row["PREV_XC"] 
            x1 = row["XC"] 
            y0 = row["PREV_YC"] 
            y1 = row["YC"] 
            x_interp = x0 + (x1 - x0) * ((t - t0) / (t1 - t0)) 
            y_interp = y0 + (y1 - y0) * ((t - t0) / (t1 - t0)) 
            xlist.append(x_interp) 
            ylist.append(y_interp) 
            break 
    
    return xlist, ylist

# test the function 
calc_line_path(0.38, dfr)

([np.float64(1.0), np.float64(2.0), np.float64(3.0), np.float64(4.0)],
 [np.float64(10.0), np.float64(15.0), np.float64(25.0), np.float64(26.0)])

## create_line_definition 

In [39]:
def create_line_definition(df_pos, metric_col, fps = 10, start_time = 0): 

    # get the min frame id 
    first_frame = df_pos["frame_id"].min() 

    # transform the frame ids 
    frame_ids = (df_pos["frame_id"] - first_frame + 1).tolist() 

    # create the line definition 
    line_def = {
        "start_time": start_time, 
        "fps": fps, 
        "path": {
            "frame_id": frame_ids, 
            "xc": [(frame - 1) / fps for frame in frame_ids], 
            "yc": df_pos[metric_col].tolist() 
        }
    } 

    return line_def 

## LineObject 

In [40]:
# class for chart lines 
class LineObject:

    def __init__(self, id, lines = [], lcolor = "blue"): 

        # initial parameters 
        self.id = id 
        self.lcolor = lcolor 
        # self.keyframes = [] 
        self.lines = lines 

        # default parameters 
        self.defaults = { 
            "alpha": 1 
        } 
    
    # draw the route at time t 
    def draw(self, ax, t = 0): 

        # calculate line path at time t 
        dfr = create_lines_df(self.lines) 
        xlist, ylist = calc_line_path(t, dfr) 

        # draw the line with markers
        ax.plot(
            xlist, 
            ylist, 
            linewidth = 2, 
            color = self.lcolor, 
            alpha = self.defaults["alpha"], 
            marker = 'o', 
            markersize = 4, 
            markerfacecolor = self.lcolor, 
            zorder = 15 
        ) 


# # create the chart animation 
# ca = ChartAnimation() 

# # add a line object 
# ca.add_object(LineObject(
#     id = "line_1", 
#     lines = [
#         {
#             "start_time": 0, 
#             "fps": 10, 
#             "path": {
#                 "frame_id": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 
#                 "xc": [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0], 
#                 "yc": [0.1, 0.12, 0.15, 0.2, 0.3, 0.5, 0.7, 0.85, 0.9, 1.0] 
#             }
#         }
#     ], 
#     lcolor = "blue" 
# )) 

# # draw and save the chart 
# ca.save_image(t = 0.85, save_file = f"{results_path}//chart_test_image.png") 
# plt.close() 

# # create and save the animation 
# ca.create_animation(
#     end_time = 1.0, 
#     fps = 12, 
#     save_file = f"{results_path}//chart_test_animation.mp4"
# ) 

# Chart Examples 

## Example 1 

In [41]:
# load the play data 
pdl = PlayDataLoader(
    game_id = 2023120400, 
    play_id = 2958, 
    df_meta = df_meta 
) 

# create the chart animation 
ca = ChartAnimation(
    xmax = 7, 
    ymax = 25, 
    plot_title = "Receiver vs Defender Speed",
    x_label = "Time (seconds)",
    y_label = "Speed (mph)"
) 

# players to iterate through 
players = [
    [55925, "red"], 
    [44849, "blue"] 
] 

# add a line object for each player 
for p in players:

    # get the player data 
    df_player = pdl.get_player_data(nfl_id = p[0]) 

    # create and add the line object 
    ca.add_object(LineObject(
        id = f"line_{p[0]}", 
        lines = [create_line_definition(df_player, metric_col="speed_mph", fps = 10, start_time = 0)], 
        lcolor = p[1] 
    )) 

# # showcase at time t 
# ca.save_image(t = 5.6, save_file = f"{results_path}//chart_test_image.png") 

# # create and save the animation 
# ca.create_animation(
#     end_time = (pdl.total_frames / 10) + 2, 
#     fps = 24, 
#     save_file = f"{results_path}//chart_speed_animation.mp4"
# ) 

## Example 2 

In [42]:
# load the play data 
pdl = PlayDataLoader(
    game_id = 2023092100, 
    play_id = 353, 
    df_meta = df_meta 
) 

# create the chart animation 
ca = ChartAnimation(
    xmax = 1.5, 
    ymin = 0, 
    ymax = 10, 
    plot_title = "Acceleration to Ball Landing Position",
    x_label = "Time (seconds)",
    y_label = "Acceleration (yd/s²)"
) 

# players to iterate through 
players = [
    [55888, "red"], 
    [47819, "blue"] 
] 

# add a line object for each player 
for p in players:

    # get the player data 
    df_player = pdl.get_player_data(nfl_id = p[0], from_frame = pdl.input_frames + 1) 

    # create and add the line object 
    ca.add_object(LineObject(
        id = f"line_{p[0]}", 
        lines = [create_line_definition(df_player, metric_col="accel_to_ball", fps = 10, start_time = pdl.input_frames / 10)], 
        lcolor = p[1] 
    )) 

# # showcase at time t 
# ca.save_image(t = 5.6, save_file = f"{results_path}//chart_test_image.png") 

# # create and save the animation 
# ca.create_animation(
#     end_time = (pdl.total_frames / 10) + 2, 
#     fps = 24, 
#     save_file = f"{results_path}//chart_speed_animation.mp4"
# ) 